In [2]:
import json
import datetime
from dateutil.parser import parse as date_parser
from pandas import Series, DataFrame

In [3]:
f = open('201501.json', 'rb')
# print f
# i = 0
# for line in f:
#     i += 1
# print i

In [4]:
class Measurement:
    def __init__(self, name, code, allowed_units):
        self.name = name
        self.code = code
        self.allowed_units = allowed_units

class Unit:
    ''' 
    si: the value needed to multiply by to get the SI unit equivalent
    '''
    def __init__(self, name, code, si):
        self.code = code
        self.name = name
        self.si = si
    def to_si(self, val):
        return self.si * val

millivolts = Unit('millivolts', 'mV', 0.001)
centivolts = Unit('centivolts', 'cV', 0.01)
celcius_16 = Unit('celcius_16', 'C16', 1./16)
percent = Unit('percent', '%', 1.)
hours = Unit('hours', 'h', 3600.)

battery_measurement = Measurement('battery', 'B', [millivolts, centivolts])
temperature_measurement = Measurement('temperature', 'T', [celcius_16])
occupancy_measurement = Measurement('occupancy', 'O', [])
light_measurement = Measurement('light', 'L', [])
relative_humidity_measurement = Measurement('relative_humidity', 'H', [percent])
vacancy_measurment = Measurement('vacancy', 'vac', [hours])

CODE_TO_NAME = {'T': 'Temperature', 'B': 'Battery', 'O': 'Occupancy', 'L': 'Light', 'H': 'Relative Humidity', 'vac': 'Vacancy'}

# Battery  B|mV and B|cV
# Temperature T|C16
# Occupancy O
# Light L
# Relative Humidity H|%
# vacancy vac|h
SELECTED_MEASUREMENTS = [temperature_measurement, occupancy_measurement, battery_measurement, relative_humidity_measurement]

# df = DataFrame(columns=['datetime_', 'name', 'val', 'units'])
data = []
for i, line in enumerate(f):
    datum = json.loads(line)
    datetime_ = date_parser(datum[0])
    sensor = datum[2]
    sensor_id = sensor["@"]
    loop_id = sensor["+"] if '+' in sensor else None
    measurements = {}
    for key, val in sensor.iteritems():
        if key in ['@', '+']:
            continue
        measurements[key] = val
    for key, val in measurements.iteritems():
        key_split = key.split('|')
        assert len(key_split) in [1, 2], 'Invalid measurement key: {}'.format(key)
        measurement_code = key_split[0]
#         is_selected = False
#         for measurement in SELECTED_MEASUREMENTS:
#             if measurement_code == measurement.code:
#                 is_selected = True
#         if not is_selected:
#             continue
        units = key_split[1] if len(key_split) == 2 else None
        x = {
            'sensor': sensor_id,
            'loop_id': loop_id,
            'datetime': datetime_,
            'measurement_type': measurement_code,
            'val': val,
            'units': units,
        }
#         df = df.append(x, ignore_index=True)
        data.append(x)
        
#     if i >= 10:
#         break

In [5]:
df = DataFrame(data=data)

In [6]:
df

,datetime,loop_id,measurement_type,sensor,units,val
0,2015-01-01 00:00:43+00:00,2,vac,0a45,h,9
1,2015-01-01 00:00:43+00:00,2,T,0a45,C16,201
2,2015-01-01 00:00:43+00:00,2,L,0a45,None,0
3,2015-01-01 00:01:13+00:00,NaN,T,819c,C16,71
4,2015-01-01 00:01:13+00:00,NaN,L,819c,None,5
5,2015-01-01 00:01:13+00:00,NaN,B,819c,cV,256
6,2015-01-01 00:01:19+00:00,4,vC,414a,%,50
7,2015-01-01 00:01:19+00:00,4,tT,414a,C,7
8,2015-01-01 00:01:19+00:00,4,v,414a,%,0
9,2015-01-01 00:01:19+00:00,4,vac,414a,h,3


In [7]:
UNIT_TO_SI = {}
MEASUREMENTS = [battery_measurement, temperature_measurement, occupancy_measurement, light_measurement, relative_humidity_measurement, vacancy_measurment]

for measurement in MEASUREMENTS:
    for unit in measurement.allowed_units:
        UNIT_TO_SI[unit.code] = unit.si

def to_si(unit_code):
    if unit_code in UNIT_TO_SI:
        return UNIT_TO_SI[unit_code]
    return None

In [8]:
df['si'] = df['units'].map(to_si)

In [9]:
df

,datetime,loop_id,measurement_type,sensor,units,val,si
0,2015-01-01 00:00:43+00:00,2,vac,0a45,h,9,3600.0000
1,2015-01-01 00:00:43+00:00,2,T,0a45,C16,201,0.0625
2,2015-01-01 00:00:43+00:00,2,L,0a45,None,0,NaN
3,2015-01-01 00:01:13+00:00,NaN,T,819c,C16,71,0.0625
4,2015-01-01 00:01:13+00:00,NaN,L,819c,None,5,NaN
5,2015-01-01 00:01:13+00:00,NaN,B,819c,cV,256,0.0100
6,2015-01-01 00:01:19+00:00,4,vC,414a,%,50,1.0000
7,2015-01-01 00:01:19+00:00,4,tT,414a,C,7,NaN
8,2015-01-01 00:01:19+00:00,4,v,414a,%,0,1.0000
9,2015-01-01 00:01:19+00:00,4,vac,414a,h,3,3600.0000


In [10]:
df['val_si'] = df['val'] * df['si']

In [11]:
df

,datetime,loop_id,measurement_type,sensor,units,val,si,val_si
0,2015-01-01 00:00:43+00:00,2,vac,0a45,h,9,3600.0000,32400.0000
1,2015-01-01 00:00:43+00:00,2,T,0a45,C16,201,0.0625,12.5625
2,2015-01-01 00:00:43+00:00,2,L,0a45,None,0,NaN,NaN
3,2015-01-01 00:01:13+00:00,NaN,T,819c,C16,71,0.0625,4.4375
4,2015-01-01 00:01:13+00:00,NaN,L,819c,None,5,NaN,NaN
5,2015-01-01 00:01:13+00:00,NaN,B,819c,cV,256,0.0100,2.5600
6,2015-01-01 00:01:19+00:00,4,vC,414a,%,50,1.0000,50.0000
7,2015-01-01 00:01:19+00:00,4,tT,414a,C,7,NaN,NaN
8,2015-01-01 00:01:19+00:00,4,v,414a,%,0,1.0000,0.0000
9,2015-01-01 00:01:19+00:00,4,vac,414a,h,3,3600.0000,10800.0000


In [12]:
df[(df['measurement_type'] == 'T') & (df['sensor'] == '0a45')]


,datetime,loop_id,measurement_type,sensor,units,val,si,val_si
1,2015-01-01 00:00:43+00:00,2,T,0a45,C16,201,0.0625,12.5625
38,2015-01-01 00:04:43+00:00,4,T,0a45,C16,201,0.0625,12.5625
79,2015-01-01 00:11:43+00:00,0,T,0a45,C16,201,0.0625,12.5625
97,2015-01-01 00:14:43+00:00,2,T,0a45,C16,201,0.0625,12.5625
120,2015-01-01 00:17:43+00:00,4,T,0a45,C16,201,0.0625,12.5625
164,2015-01-01 00:24:43+00:00,6,T,0a45,C16,201,0.0625,12.5625
248,2015-01-01 00:40:43+00:00,4,T,0a45,C16,201,0.0625,12.5625
287,2015-01-01 00:45:43+00:00,6,T,0a45,C16,201,0.0625,12.5625
299,2015-01-01 00:47:43+00:00,0,T,0a45,C16,201,0.0625,12.5625
325,2015-01-01 00:50:43+00:00,2,T,0a45,C16,201,0.0625,12.5625


In [13]:
df[(df['sensor'] == 'f1c6') & (df['measurement_type'] == 'T') & (df['datetime'] > date_parser('2015-01-01 01:00')) & (df['datetime'] < date_parser('2015-01-01 02:00'))]

,datetime,loop_id,measurement_type,sensor,units,val,si,val_si
437,2015-01-01 01:07:53+00:00,2,T,f1c6,C16,197,0.0625,12.3125
487,2015-01-01 01:17:53+00:00,6,T,f1c6,C16,197,0.0625,12.3125
582,2015-01-01 01:32:53+00:00,2,T,f1c6,C16,196,0.0625,12.2500
692,2015-01-01 01:47:53+00:00,4,T,f1c6,C16,197,0.0625,12.3125
744,2015-01-01 01:55:53+00:00,6,T,f1c6,C16,197,0.0625,12.3125


In [14]:
df


,datetime,loop_id,measurement_type,sensor,units,val,si,val_si
0,2015-01-01 00:00:43+00:00,2,vac,0a45,h,9,3600.0000,32400.0000
1,2015-01-01 00:00:43+00:00,2,T,0a45,C16,201,0.0625,12.5625
2,2015-01-01 00:00:43+00:00,2,L,0a45,None,0,NaN,NaN
3,2015-01-01 00:01:13+00:00,NaN,T,819c,C16,71,0.0625,4.4375
4,2015-01-01 00:01:13+00:00,NaN,L,819c,None,5,NaN,NaN
5,2015-01-01 00:01:13+00:00,NaN,B,819c,cV,256,0.0100,2.5600
6,2015-01-01 00:01:19+00:00,4,vC,414a,%,50,1.0000,50.0000
7,2015-01-01 00:01:19+00:00,4,tT,414a,C,7,NaN,NaN
8,2015-01-01 00:01:19+00:00,4,v,414a,%,0,1.0000,0.0000
9,2015-01-01 00:01:19+00:00,4,vac,414a,h,3,3600.0000,10800.0000


In [15]:
x = df[(df['measurement_type'] == 'T') & (df['sensor'] == '0a45')]


In [16]:
y = [val for val in x.T.to_dict().values()]

In [17]:
y

[{'datetime': Timestamp('2015-01-01 00:00:43+0000', tz='UTC'),
  'loop_id': 2.0,
  'measurement_type': u'T',
  'sensor': u'0a45',
  'si': 0.0625,
  'units': u'C16',
  'val': 201,
  'val_si': 12.5625},
 {'datetime': Timestamp('2015-01-20 01:02:31+0000', tz='UTC'),
  'loop_id': 0.0,
  'measurement_type': u'T',
  'sensor': u'0a45',
  'si': 0.0625,
  'units': u'C16',
  'val': 257,
  'val_si': 16.0625},
 {'datetime': Timestamp('2015-01-27 13:35:00+0000', tz='UTC'),
  'loop_id': 2.0,
  'measurement_type': u'T',
  'sensor': u'0a45',
  'si': 0.0625,
  'units': u'C16',
  'val': 249,
  'val_si': 15.5625},
 {'datetime': Timestamp('2015-01-16 00:27:21+0000', tz='UTC'),
  'loop_id': 4.0,
  'measurement_type': u'T',
  'sensor': u'0a45',
  'si': 0.0625,
  'units': u'C16',
  'val': 271,
  'val_si': 16.9375},
 {'datetime': Timestamp('2015-01-12 05:28:33+0000', tz='UTC'),
  'loop_id': 6.0,
  'measurement_type': u'T',
  'sensor': u'0a45',
  'si': 0.0625,
  'units': u'C16',
  'val': 257,
  'val_si': 16.06

In [18]:
from measurements.models import Measurement


ImportError: No module named measurements.models